# 회귀분석 완벽 프레임 v2 - 시험용 🔥
## 모든 기출 유형 대응! (RMSE, MAE, MAPE, MSE, R² 모두 포함)

## 1. 필수 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error  # MAPE
)
import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (있으면)
try:
    from dmba import regressionSummary
    HAS_DMBA = True
except:
    HAS_DMBA = False
    print("dmba 라이브러리가 없습니다. sklearn metrics를 사용합니다.")

## 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로드 (⭐ 파일명 수정 필수!)
df = pd.read_csv('데이터파일.csv')

print("[데이터 정보]")
df.info()

print("\n[결측치 확인]")
print(df.isnull().sum())

print("\n[데이터 샘플]")
print(df.head())

In [ ]:
# 변수 타입 자동 분류
print("[변수 타입 분류]")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"범주형 변수 ({len(categorical_cols)}개): {categorical_cols}")
print(f"수치형 변수 ({len(numerical_cols)}개): {numerical_cols}")

In [ ]:
# 기초 통계량
print("\n[종속변수 기초 통계]")
print(df['종속변수명'].describe())  # ⭐ 종속변수명 수정!

## 3. 데이터 전처리

In [ ]:
# 결측치 처리 (필요시)
# df = df.dropna()

# 독립변수/종속변수 분리 (⭐ 여기 수정 필수!)
outcome = '종속변수명'  # ← 종속변수 이름 (예: price, charges, streams)
predictors = [col for col in df.columns 
              if col not in [outcome, '제외할변수1', '제외할변수2']]

X = df[predictors]
y = df[outcome]

print(f"독립변수 개수: {len(predictors)}")
print(f"데이터 크기: {X.shape}")

In [ ]:
# 범주형 변수 원-핫 인코딩
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

if len(categorical_features) > 0:
    # 회귀분석은 drop_first=True 권장 (다중공선성 방지)
    X = pd.get_dummies(X, columns=categorical_features, drop_first=True)
    print(f"\n원-핫 인코딩 후 변수 개수: {X.shape[1]}")
    print(f"변수 목록 (처음 10개): {X.columns.tolist()[:10]}")

## 4. 데이터 분리 (두 가지 방법)

In [ ]:
# 방법 1: train_test_split 사용 (일반적)
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=1
)

print(f"학습 데이터: {train_X.shape}")
print(f"테스트 데이터: {test_X.shape}")

In [ ]:
# 방법 2: 인덱스 기반 분리 (기출문제 스타일)
# 예: 2023년 문제 "0~899번까지 학습, 나머지 테스트"

# train_X = X.iloc[:900]
# train_y = y.iloc[:900]
# test_X = X.iloc[900:]
# test_y = y.iloc[900:]

# print(f"학습 데이터: {train_X.shape}")
# print(f"테스트 데이터: {test_X.shape}")

## 5. 데이터 표준화 (Ridge/Lasso/ElasticNet용)

In [ ]:
# 수치형 변수 표준화
# Ridge/Lasso/ElasticNet은 표준화 필수!
# Linear Regression은 선택사항 (기출에서는 보통 안 함)

scaler = StandardScaler()

# 표준화할 변수들 (수치형만)
numerical_features = train_X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 학습 데이터로 fit, 학습/테스트 모두 transform
train_X_scaled = train_X.copy()
test_X_scaled = test_X.copy()

if len(numerical_features) > 0:
    train_X_scaled[numerical_features] = scaler.fit_transform(train_X[numerical_features])
    test_X_scaled[numerical_features] = scaler.transform(test_X[numerical_features])
    print(f"표준화 완료: {len(numerical_features)}개 변수")
else:
    print("표준화할 수치형 변수가 없습니다.")

## 6. 평가 함수 정의 (⭐ 기출 대응!)

In [ ]:
def evaluate_regression(y_true, y_pred, model_name="Model"):
    """
    회귀 모델 평가 - 모든 기출 지표 포함!
    
    기출 지표:
    - RMSE: 2023년
    - MAE: 2021년, 2022년
    - MAPE: 2024년
    - MSE: 2024년 (cross_val_score용)
    - R²: 항상 유용
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100  # %로 표시
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{'='*60}")
    print(f"📊 {model_name} 성과")
    print(f"{'='*60}")
    print(f"RMSE:  {rmse:>12,.4f}  ← 2023년 기출")
    print(f"MAE:   {mae:>12,.4f}  ← 2021, 2022년 기출")
    print(f"MAPE:  {mape:>12.2f}%  ← 2024년 기출")
    print(f"MSE:   {mse:>12,.4f}  ← 2024년 기출 (CV용)")
    print(f"R²:    {r2:>12.4f}")
    print(f"{'='*60}")
    
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'MSE': mse, 'R2': r2}

## 7. 모델 학습 및 평가

### 7-1. Linear Regression (선형회귀)

In [ ]:
print("="*60)
print("📊 1. Linear Regression (기본 선형회귀)")
print("="*60)

# 모델 학습 (표준화 안 한 데이터 사용 - 기출 스타일)
lr = LinearRegression()
lr.fit(train_X, train_y)

# 예측
lr_pred_train = lr.predict(train_X)
lr_pred_test = lr.predict(test_X)

# 학습 데이터 성과
print("\n[학습 데이터 성과]")
if HAS_DMBA:
    regressionSummary(train_y, lr_pred_train)
else:
    lr_train_results = evaluate_regression(train_y, lr_pred_train, "Linear Regression - Train")

# 테스트 데이터 성과
print("\n[테스트 데이터 성과]")
if HAS_DMBA:
    regressionSummary(test_y, lr_pred_test)
else:
    lr_test_results = evaluate_regression(test_y, lr_pred_test, "Linear Regression - Test")

In [ ]:
# 회귀계수 확인
print("\n[회귀계수 Top 10]")
coef_df = pd.DataFrame({
    'Variable': train_X.columns,
    'Coefficient': lr.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_df.head(10).to_string(index=False))
print(f"\nIntercept: {lr.intercept_:.4f}")

### 7-2. Ridge Regression (alpha 튜닝)

In [ ]:
print("="*60)
print("📊 2. Ridge Regression (L2 규제)")
print("="*60)

# 하이퍼파라미터 튜닝 (RandomizedSearchCV)
# 기출 스타일: np.linspace(0.0001, 1000, num=500)
param_dist = {
    'alpha': np.linspace(0.0001, 1000, num=500)
}

ridge = Ridge()

# 기출에서 자주 쓰는 scoring 옵션들
# MAE: neg_mean_absolute_error (2021, 2022)
# MSE: neg_mean_squared_error (2024)
# RMSE: neg_root_mean_squared_error

random_search_ridge = RandomizedSearchCV(
    ridge,
    param_distributions=param_dist,
    n_iter=200,  # 기출에서 자주 200번
    cv=10,  # 10겹 교차검증
    scoring='neg_mean_absolute_error',  # ← 기출 스타일: MAE
    random_state=1,
    n_jobs=-1
)

print("\n⏳ Ridge 하이퍼파라미터 튜닝 중... (10-Fold CV, MAE)")
random_search_ridge.fit(train_X_scaled, train_y)

print(f"\n✅ 최적 alpha: {random_search_ridge.best_params_['alpha']:.6f}")
print(f"✅ 최고 CV MAE: {-random_search_ridge.best_score_:.4f}")

# 최적 모델로 예측
ridge_best = random_search_ridge.best_estimator_
ridge_pred_train = ridge_best.predict(train_X_scaled)
ridge_pred_test = ridge_best.predict(test_X_scaled)

# 테스트 데이터 성과
print("\n[테스트 데이터 성과]")
ridge_test_results = evaluate_regression(test_y, ridge_pred_test, "Ridge Regression - Test")

### 7-3. Lasso Regression (alpha 튜닝)

In [ ]:
print("="*60)
print("📊 3. Lasso Regression (L1 규제)")
print("="*60)

# 하이퍼파라미터 튜닝
param_dist = {
    'alpha': np.linspace(0.0001, 1000, num=500)
}

lasso = Lasso(max_iter=10000)  # 수렴을 위해 max_iter 증가

random_search_lasso = RandomizedSearchCV(
    lasso,
    param_distributions=param_dist,
    n_iter=200,
    cv=10,
    scoring='neg_mean_absolute_error',
    random_state=1,
    n_jobs=-1
)

print("\n⏳ Lasso 하이퍼파라미터 튜닝 중... (10-Fold CV, MAE)")
random_search_lasso.fit(train_X_scaled, train_y)

print(f"\n✅ 최적 alpha: {random_search_lasso.best_params_['alpha']:.6f}")
print(f"✅ 최고 CV MAE: {-random_search_lasso.best_score_:.4f}")

# 최적 모델로 예측
lasso_best = random_search_lasso.best_estimator_
lasso_pred_train = lasso_best.predict(train_X_scaled)
lasso_pred_test = lasso_best.predict(test_X_scaled)

# 테스트 데이터 성과
print("\n[테스트 데이터 성과]")
lasso_test_results = evaluate_regression(test_y, lasso_pred_test, "Lasso Regression - Test")

In [ ]:
# Lasso 변수 선택 결과
print("\n[Lasso 변수 선택 결과]")
lasso_coef = pd.DataFrame({
    'Variable': train_X_scaled.columns,
    'Coefficient': lasso_best.coef_
})

# 0이 아닌 계수만
selected_vars = lasso_coef[lasso_coef['Coefficient'] != 0].sort_values('Coefficient', key=abs, ascending=False)
print(f"선택된 변수 개수: {len(selected_vars)} / {len(lasso_coef)}")
print("\n선택된 변수 Top 10:")
print(selected_vars.head(10).to_string(index=False))

### 7-4. ElasticNet Regression (alpha & l1_ratio 튜닝)

In [ ]:
print("="*60)
print("📊 4. ElasticNet Regression (L1 + L2 규제)")
print("="*60)

# 하이퍼파라미터 튜닝 (alpha & l1_ratio 둘 다!)
param_dist = {
    'alpha': np.linspace(0.0001, 1000, num=500),
    'l1_ratio': np.linspace(0, 1, num=500)  # 0: Ridge, 1: Lasso
}

elastic = ElasticNet(max_iter=10000)

random_search_elastic = RandomizedSearchCV(
    elastic,
    param_distributions=param_dist,
    n_iter=200,
    cv=10,
    scoring='neg_mean_absolute_error',
    random_state=1,
    n_jobs=-1
)

print("\n⏳ ElasticNet 하이퍼파라미터 튜닝 중... (10-Fold CV, MAE)")
random_search_elastic.fit(train_X_scaled, train_y)

print(f"\n✅ 최적 alpha: {random_search_elastic.best_params_['alpha']:.6f}")
print(f"✅ 최적 l1_ratio: {random_search_elastic.best_params_['l1_ratio']:.6f}")
print(f"✅ 최고 CV MAE: {-random_search_elastic.best_score_:.4f}")

# 최적 모델로 예측
elastic_best = random_search_elastic.best_estimator_
elastic_pred_train = elastic_best.predict(train_X_scaled)
elastic_pred_test = elastic_best.predict(test_X_scaled)

# 테스트 데이터 성과
print("\n[테스트 데이터 성과]")
elastic_test_results = evaluate_regression(test_y, elastic_pred_test, "ElasticNet Regression - Test")

## 8. 전체 모델 성과 비교 ⭐

In [ ]:
print("="*80)
print("🏆 전체 모델 성과 비교 (테스트 데이터)")
print("="*80)

comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge', 'Lasso', 'ElasticNet'],
    'RMSE': [
        lr_test_results['RMSE'],
        ridge_test_results['RMSE'],
        lasso_test_results['RMSE'],
        elastic_test_results['RMSE']
    ],
    'MAE': [
        lr_test_results['MAE'],
        ridge_test_results['MAE'],
        lasso_test_results['MAE'],
        elastic_test_results['MAE']
    ],
    'MAPE (%)': [
        lr_test_results['MAPE'],
        ridge_test_results['MAPE'],
        lasso_test_results['MAPE'],
        elastic_test_results['MAPE']
    ],
    'MSE': [
        lr_test_results['MSE'],
        ridge_test_results['MSE'],
        lasso_test_results['MSE'],
        elastic_test_results['MSE']
    ],
    'R²': [
        lr_test_results['R2'],
        ridge_test_results['R2'],
        lasso_test_results['R2'],
        elastic_test_results['R2']
    ]
})

print("\n", comparison_df.to_string(index=False))

# 최고 성능 모델 (MAE 기준)
best_model_idx = comparison_df['MAE'].idxmin()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_mae = comparison_df.loc[best_model_idx, 'MAE']

print(f"\n✅ 최고 성능 모델 (MAE 기준): {best_model_name} (MAE: {best_mae:.4f})")

In [ ]:
# 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['RMSE', 'MAE', 'MAPE (%)', 'R²']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx // 2, idx % 2]
    
    if metric == 'R²':
        # R²는 높을수록 좋음
        bars = ax.barh(comparison_df['Model'], comparison_df[metric], color=color, alpha=0.7)
        best_idx = comparison_df[metric].idxmax()
    else:
        # 나머지는 낮을수록 좋음
        bars = ax.barh(comparison_df['Model'], comparison_df[metric], color=color, alpha=0.7)
        best_idx = comparison_df[metric].idxmin()
    
    # 최고 성능 강조
    bars[best_idx].set_alpha(1.0)
    bars[best_idx].set_edgecolor('black')
    bars[best_idx].set_linewidth(2)
    
    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 9. 교차검증 (5-Fold or 10-Fold) ⭐

In [ ]:
print("="*80)
print("📊 교차검증 성과 비교 (전체 데이터 사용)")
print("="*80)

# 기출에서 자주 쓰는 평가 지표들
scoring_metrics = {
    'MAE': 'neg_mean_absolute_error',  # 2021, 2022
    'MSE': 'neg_mean_squared_error',   # 2024
    'RMSE': 'neg_root_mean_squared_error',  # 2023
}

models = {
    'Linear Regression': LinearRegression(),
    'Ridge': ridge_best,
    'Lasso': lasso_best,
    'ElasticNet': elastic_best
}

cv_results = []

for model_name, model in models.items():
    result = {'Model': model_name}
    
    # 데이터 선택 (Linear는 원본, 나머지는 표준화)
    if model_name == 'Linear Regression':
        X_use = X
    else:
        X_use = pd.DataFrame(
            scaler.fit_transform(X[numerical_features]) if len(numerical_features) > 0 else X,
            columns=X.columns
        )
    
    # 각 평가 지표로 교차검증
    for metric_name, scoring in scoring_metrics.items():
        scores = cross_val_score(model, X_use, y, cv=5, scoring=scoring)
        # negative를 positive로 변환
        result[f'{metric_name} (5-Fold CV)'] = -scores.mean()
        result[f'{metric_name} Std'] = scores.std()
    
    cv_results.append(result)

cv_df = pd.DataFrame(cv_results)
print("\n[5-Fold Cross Validation 결과]")
print(cv_df.to_string(index=False))

In [ ]:
# 🎯 기출 스타일: 특정 지표만 출력
print("\n" + "="*80)
print("🎯 기출 형식 출력 (주요 지표만)")
print("="*80)

for model_name in models.keys():
    model_data = cv_df[cv_df['Model'] == model_name].iloc[0]
    
    print(f"\n{model_name}:")
    print(f"  - 5-Fold CV MAE:  {model_data['MAE (5-Fold CV)']:.4f} (±{model_data['MAE Std']:.4f})")
    print(f"  - 5-Fold CV RMSE: {model_data['RMSE (5-Fold CV)']:.4f} (±{model_data['RMSE Std']:.4f})")
    print(f"  - 5-Fold CV MSE:  {model_data['MSE (5-Fold CV)']:.4f} (±{model_data['MSE Std']:.4f})")

## 10. 최종 결론 📝

In [ ]:
print("="*80)
print("🎯 최종 결론 (기출 대응 완료!)")
print("="*80)

print(f"""
✅ 분석 완료!

📊 최고 성능 모델: {best_model_name}

🎯 테스트 데이터 성과:
   - RMSE: {comparison_df.loc[best_model_idx, 'RMSE']:.4f}  ← 2023년 기출 지표
   - MAE:  {comparison_df.loc[best_model_idx, 'MAE']:.4f}  ← 2021, 2022년 기출 지표
   - MAPE: {comparison_df.loc[best_model_idx, 'MAPE (%)']:.2f}%  ← 2024년 기출 지표
   - MSE:  {comparison_df.loc[best_model_idx, 'MSE']:.4f}  ← 2024년 기출 지표 (CV용)
   - R²:   {comparison_df.loc[best_model_idx, 'R²']:.4f}

💡 하이퍼파라미터 튜닝 결과:
   - Ridge 최적 alpha: {random_search_ridge.best_params_['alpha']:.6f}
   - Lasso 최적 alpha: {random_search_lasso.best_params_['alpha']:.6f}
   - ElasticNet 최적 alpha: {random_search_elastic.best_params_['alpha']:.6f}
   - ElasticNet 최적 l1_ratio: {random_search_elastic.best_params_['l1_ratio']:.6f}

🎉 모든 기출 유형 대응 완료!
""")